# Exercise 1: NYC Taxi Dataset Analysis with Unix Tools and SQL

Your name: ___Prasanna Bhat___

This assignment is split in three parts:
 1. In the first part, we will analyze a large data set with UNIX tools.
 2. Next, we will see how the analysis we did would be done in SQL.
 3. Lastly, we included a refresher on more advanced SQL.

### Introduction

As you have seen in the lecture, **UNIX tools** are a handy way to quickly get some insights into data sets. They are **available on almost all systems and don't require any setup**, which is a huge advantage if you think about it! I still use the Unix tools we discussed almost every day.

However, they will only get you so far. That's the reason why we will also look into other, **more practical and specialized** ways to analyze large data sets after the first half of the assignment. For **relational datasets**, like the one we have here, you would use some sort of database. **DuckDB** is especially suitable here because it is a very modern implementation and allows you to run queries directly on raw files without needing to import them into the database. This makes it very handy for this task. You will quickly realize how different and probably more straight-forward it will be. You can use the results from the second part to cross-check with your results from the first-part.

Lastly, we will give you some tasks that require the use of more involved SQL features. This should serve as a refresher for you. If you never touched SQL, it might be a bit trickier, but surely not impossible! You can always reach out for help.

### Assignment Structure
We will use Jupyter notebooks throughout most of the exercises. The reason for this is that this tool is oftentimes used in the data science / data engineering space.

Also, particularly in this assignment, we will use bash commands that we run from this notebook. We do this by marking the whole cell for executing with bash via the `%%bash` magic at the beginning. An alternative is to use `%bash` in front of a single line or use `!` followed by the command.
The same applies for SQL code. In this code, we will use the so-called sql "line magic", that is, after setting up the proper config, you can run SQL in DuckDB by just starting a line with `%sql` followed by the SQL string. If you want to write multi-line SQL, you can end a line with `\`, which means that the line will be concatenated with the following line. Be careful with comments if you are using the trick with `\`!

The following Python packages are required for this assignment:
```
pandas
pyarrow
duckdb
jupysql
duckdb-engine
matplotlib
pillow
```


### The Dataset
We will use a data set that contains taxi drives from New York, which is provided for free by the city of New York. Interestingly, this data set is used in many research works because it (a) is easy to work with (b) contains real-life data and (c) has a decent size. For this exercise, we will only use the data from August 2021.

Documentation of this data set is available at <https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page>. Specifically, have a look at <https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf>.
When you look on the website, you will see several mentions of the "Parquet format". We will not use this for now, but we will learn about it later in the course. For now, we will first convert it to CSV, so that you can manually inspect the file.


In [2]:
import pandas as pd

# Download New York Taxi trip data set and convert it to CSV
#df = pd.read_parquet('yellow_tripdata_2021-08.parquet')
df = pd.read_csv('2021_Yellow_Taxi_Trip_Data.csv', low_memory=False)

In [3]:
print(df.head(20))

    Unnamed: 0  VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
0            0         2  2021-08-01 00:04:06   2021-08-01 00:26:07   
1            1         1  2021-08-01 00:34:14   2021-08-01 00:43:02   
2            2         1  2021-08-01 00:43:53   2021-08-01 00:55:04   
3            3         1  2021-08-01 00:53:19   2021-08-01 00:55:34   
4            4         1  2021-08-01 00:58:54   2021-08-01 01:11:23   
5            5         2  2021-08-01 00:11:59   2021-08-01 00:17:15   
6            6         2  2021-08-01 00:28:29   2021-08-01 00:48:37   
7            7         1  2021-08-01 00:21:11   2021-08-01 00:32:59   
8            8         1  2021-08-01 00:37:38   2021-08-01 00:58:26   
9            9         2  2021-08-01 00:08:46   2021-08-01 00:30:40   
10          10         2  2021-08-01 00:37:07   2021-08-01 00:49:50   
11          11         2  2021-08-01 00:54:32   2021-08-01 01:00:17   
12          12         2  2021-08-01 18:15:50   2021-08-01 18:30:54   
13    

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2788757 entries, 0 to 2788756
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Example

#### Example Q: How many entries does the "2021 Yellow Taxi Trip Data" have?

In [4]:
%%bash

# Explanation: tail -n +2 can be used to skip the header of the csv file. wc -l counts the lines in the output stream.
tail -n +2 "2021_Yellow_Taxi_Trip_Data.csv" | wc -l

2788757


--------

# Part 1: UNIX tools


## Task
Try to answer the questions in the quiz. You should provide the command that you used to come to your answer together with a short explanation. The command should be a "one-liner" that outputs the correct result. You can either work from the terminal on your computer and copy-paste the solutions into the notebook cells or directly run bash commands in the notebook (see example).

## 1) Outliers

First, we would like to understand more about the extreme values of the data set. An early understanding of this helps further down the road, i.e. if some of the extreme values look unreasonable, they should be filtered out before further processing the data set. Sometimes, they even have to be corrected!


#### Q1.1: What is the trip that had the most expensive `total_amount`? Comment on the value, i.e. is it reasonable?

In [6]:
%%bash
echo "The most expensive trip: $(cat '2021_Yellow_Taxi_Trip_Data.csv' | sort -t, -k18,18nr | head -n 1)"


The most expensive trip: 661923,1,2021-08-08 18:20:39,2021-08-08 19:11:21,4.0,12.4,1.0,N,50,188,4,43.0,2.5,0.5,0.0,913.01,0.3,959.31,2.5,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2788757 entries, 0 to 2788756
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   VendorID               int64  
 2   tpep_pickup_datetime   object 
 3   tpep_dropoff_datetime  object 
 4   passenger_count        float64
 5   trip_distance          float64
 6   RatecodeID             float64
 7   store_and_fwd_flag     object 
 8   PULocationID           int64  
 9   DOLocationID           int64  
 10  payment_type           int64  
 11  fare_amount            float64
 12  extra                  float64
 13  mta_tax                float64
 14  tip_amount             float64
 15  tolls_amount           float64
 16  improvement_surcharge  float64
 17  total_amount           float64
 18  congestion_surcharge   float64
 19  airport_fee            float64
dtypes: float64(12), int64(5), object(3)
memory usage: 425.5+ MB


#### Q1.2: What's the distance of the longest and shortest trip in kilometers? Comment on the value, i.e. is it reasonable? Try to come up with a one-liner that outputs both numbers in one line!  
_Note: The distances in the file are in miles. Try to convert them into kilometers inside your command._

In [13]:
%%bash
echo "Distance of longest and shortest trip: $(cat '2021_Yellow_Taxi_Trip_Data.csv' |cut -d',' -f6| sort -nr | sed -n '1p*1.609;$p')"
#sed - stream editor used to select a particular row

sed: -e expression #1, char 3: extra characters after command


Distance of longest and shortest trip: 


#### Q1.3: What is the maximum number of passengers on a single taxi ride? Comment on the value, i.e. is it reasonable?

In [5]:
%%bash
cat '2021_Yellow_Taxi_Trip_Data.csv'| cut -d ',' -f5  | sort -nr | head -n 1 

112.0


#### Q1.4: How many entries do *not* belong in August 2021, i.e. when a trip neither has pickup_datetime nor dropoff_datetime in August 2021?

In [45]:
%%bash
awk -F, 'NR > 1 {if ($3 !~ /^2021-08/ && $4 !~ /^2021-08/) {count++}}END { print count }' 2021_Yellow_Taxi_Trip_Data.csv

476


In [8]:
%%bash 
awk -F, '(NR>1 && $3 !~ /^2021-08/ && $4 !~ /^2021-08/)' 2021_Yellow_Taxi_Trip_Data.csv | wc -l

476


In [34]:
%%bash
cat '2021_Yellow_Taxi_Trip_Data.csv'|cut -d',' -f4|head -n 2

tpep_dropoff_datetime
2021-08-01 00:26:07


## 2) Missing Values

Next, we'll look at missing values. Missing values might be indicated in various different ways, e.g. NULL, NIL, None, Undefined, -1, empty strings, ...

The column `passenger_count` has quite a lot of missing values.

#### Q2.1: How many missing values are there in the `passenger_count` column?

In [96]:
%%bash
awk -F',' 'NR > 1 && ($5 == "")' 2021_Yellow_Taxi_Trip_Data.csv | wc -l

# <your code here>

135590


#### Q2.2: Describe how you figured out what value is considered "empty"? Is the data set using NULL, -1 or something different? Come up with a clever way using UNIX tools!

In [95]:
%%bash
awk -F, '
NR > 1 {
    if ($5 == "") empty++
    else if ($5 == "NULL") null++
    else if ($5 == "NIL") nil++
    else if ($5 == "None") none++
    else if ($5 == "Undefined") undefined++
    else if ($5 == "-1") minus_one++
}
END {
    print "Empty:", empty+0
    print "NULL:", null+0
    print "NIL:", nil+0
    print "None:", none+0
    print "Undefined:", undefined+0
    print "-1:", minus_one+0
}' 2021_Yellow_Taxi_Trip_Data.csv


Empty: 135590
NULL: 0
NIL: 0
None: 0
Undefined: 0
-1: 0


## 3) Distributions

Another important piece of information about a data set is a notion of distribution. Data sets might be "top-heavy", i.e. a significant portion of data points are concentrated towards the higher end of the scale, or "bottom-heavy", which is just the opposite of top-heavy. When it comes to heavily skewed data distributions, it's important to understand that single metrics like a MEAN don't fully describe the "average characteristics". A property of top-heavy data sets is that the MEAN is larger than the MEDIAN because there are some large values that pull up the MEDIAN. As we will see in a minute, `passenger_count` is a bottom-heavy distribution.




Let's have a closer look at the `passenger_count` column in this exercise. Only use UNIX tools that are installed per default. No Python scripting, datamash or whatever ;)

#### Q3.1: What is the mean value for the number of passengers?

In [22]:
%%bash
awk -F, '$5>0 {sum += $5; count++} END {print sum / count}' 2021_Yellow_Taxi_Trip_Data.csv

1.48016


#### Q3.2: What is the median value for the number of passengers?

_Note: This one is surprisingly tricky. You can assume that you have the number of entries in the data set. Extra points if manage to compute it without any assumptions about the size of the data set._

In [25]:
%%bash
awk -F',' '$5 ~ /^[0-9]+$/ {print $5}' 2021_Yellow_Taxi_Trip_Data.csv | sort -n | awk '{count[NR] = $1} END {if (NR % 2) {print count[(NR + 1) / 2]} else {print (count[NR / 2] + count[NR / 2 + 1]) / 2}}'


0


In [26]:
%%bash
cut -d ',' -f5 2021_Yellow_Taxi_Trip_Data.csv | sort -n|awk '{count[NR] = $1} END {if (NR % 2) {print count[(NR + 1) / 2]} else {print (count[NR / 2] + count[NR / 2 + 1]) / 2}}'

1


#### Q3.3: What is the p95 value for passenger count?

In [1]:
%%bash
awk -F',' '$5 ~ /^[0-9]+$/ {print $5}' 2021_Yellow_Taxi_Trip_Data.csv | sort -n | awk '{count[NR] = $1} END {if (NR > 0) {print count[int(NR * 0.95 + 0.5)]}}'


In [104]:
awk -F, 'NR > 1 && $5 > 0 {print $5}' 2021_Yellow_Taxi_Trip_Data.csv | sort -n | awk 'NR==int(0.95*NR) {print; exit}'


SyntaxError: invalid decimal literal (2209686485.py, line 1)

#### Q3.4 How many distinct values (NDV) are there for the `passenger_count` column?

In [43]:
%%bash

cut -d ',' -f5 2021_Yellow_Taxi_Trip_Data.csv | sort | uniq | wc -l


14


-------
# Part 2: Beyond Bash: SQL

You might have noticed that UNIX commands can get pretty complex and are sometimes hard to understand. Also, their performance is sometimes not the best.
In the second part of this exercise you get the chance to do the same thing again, but in SQL. For this, we use DuckDB inside this notebook.

To use DuckDb, you first have to install it (see also https://duckdb.org/docs/guides/python/jupyter.html):
```
pip install duckdb
pip install jupysql
pip install duckdb-engine
pip install pandas
pip install matplotlib
```

If you installed everything correctly, the following cell should just work:

In [5]:
import duckdb
import pandas as pd

# Enable using '%sql' syntax in notebooks:
%reload_ext sql
# Initialize DuckDB
%sql duckdb:///:memory:
# Set output format options via pandas for DuckDB
pd.set_option('display.max_columns', None)

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connecting to 'duckdb:///:memory:'

In [16]:
# Test that everything works by running a test-query on the input file using DuckDB.
%sql select count(*) as cnt from '2021_Yellow_Taxi_Trip_Data.csv';

,cnt
0,2788757


#### Q4.1: What is the trip that had the most expensive `total_amount`?

In [22]:
# Answer:
%sql SELECT * FROM '2021_Yellow_Taxi_Trip_Data.csv' ORDER BY total_amount DESC LIMIT 1;

,column00,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,661923,1,2021-08-08 18:20:39,2021-08-08 19:11:21,4.0,12.4,1.0,N,50,188,4,43.0,2.5,0.5,0.0,913.01,0.3,959.31,2.5,0.0


In [ ]:
# Cell is used for grading. Do not remove.

#### Q4.2: What's the distance of the longest and shortest trip in kilometers?

In [48]:
# Answer:

%sql SELECT MAX(trip_distance)*1.609 as maximum_distance, MIN(trip_distance)*1.609 as minimum_distance FROM '2021_Yellow_Taxi_Trip_Data.csv' 

,maximum_distance,minimum_distance
0,495473.17522,0.0


In [ ]:
# Cell is used for grading. Do not remove.

#### Q4.3: What is the maximum number of passengers on a single taxi ride?

In [28]:
# Answer:

%sql select max(passenger_count) from '2021_Yellow_Taxi_Trip_Data.csv'

,max(passenger_count)
0,112.0


In [ ]:
# Cell is used for grading. Do not remove.

In [35]:
%sql PRAGMA table_info('2021_Yellow_Taxi_Trip_Data.csv');

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.CatalogException) Catalog Error: Table with name csv does not exist!
Did you mean "main.duckdb_views"?
[SQL: PRAGMA table_info('2021_Yellow_Taxi_Trip_Data.csv');]
(Background on this error at: https://sqlalche.me/e/14/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


#### Q4.4: How many entries do not belong in August 2021, i.e. when a trip neither has pickup_datetime nor dropoff_datetime in August 2021?

In [39]:
# Answer:
%sql select count(*) from '2021_Yellow_Taxi_Trip_Data.csv' where not(CAST(tpep_dropoff_datetime AS DATE) between '2021-08-01' and '2021-08-31') and not(CAST(tpep_pickup_datetime AS DATE) between '2021-08-01' and '2021-08-31') 

,count_star()
0,476


In [42]:
#another approach tried

In [43]:

%%sql 
SELECT COUNT(*)
FROM '2021_Yellow_Taxi_Trip_Data.csv'
WHERE 
date_trunc('month', CAST(tpep_dropoff_datetime AS DATE)) != '2021-08-01'
AND date_trunc('month', CAST(tpep_pickup_datetime AS DATE)) != '2021-08-01';


,count_star()
0,476


In [ ]:
# Cell is used for grading. Do not remove.

#### Q4.5: How many missing values are there in the `passenger_count` column?

In [25]:
# Answer:
%sql select count(*) from '2021_Yellow_Taxi_Trip_Data.csv' where passenger_count is NULL or passenger_count = 0
# <your code here>

,count_star()
0,208478


In [ ]:
# Cell is used for grading. Do not remove.

#### Q4.6: What are the mean, median, p95, and number of distinct values (NDV) for the number of passengers? Return all of the numbers in just one query.

In [21]:
    %%sql SELECT AVG(passenger_count) average,MEDIAN(passenger_count) median,APPROX_QUANTILE(passenger_count, 0.95) p95,COUNT(DISTINCT passenger_count) unique_count
    FROM read_csv_auto('2021_Yellow_Taxi_Trip_Data.csv') where passenger_count>0

,average,median,p95,unique_count
0,1.480158,1.0,3.999602,11


In [ ]:
# Cell is used for grading. Do not remove.

-------
# Part 3: More advanced SQL

With SQL, we can do some more advanced analytics as well. This exercise will give you the chance to refresh and brush up your SQL skills.

The following tasks require more features that SQL provides, e.g. joins, aggregations, ...

#### Q5.1: What was the day when the most money was earned by drivers?
At what day in August 2021 was the accumulated money that was spent by customers the highest?

_Note: Think about whether pickup or dropoff time should be used._

In [12]:
%%sql
SELECT CAST(tpep_dropoff_datetime AS DATE) AS dropoff_date,SUM(total_amount,Fare_amount,) AS total_amount
FROM '2021_Yellow_Taxi_Trip_Data.csv'
GROUP BY dropoff_date
ORDER BY total_amount DESC
LIMIT 1;


,dropoff_date,total_amount
0,2021-08-20,2.025402e+06


In [ ]:
# Cell is used for grading. Do not remove.

#### Q5.2: What are the most popular pickup locations in the morning (8am - 10am) and in the evening (5pm-7pm)

In [97]:
%%sql
WITH morning_ranks AS (
    SELECT 
        COUNT(*) AS pickups, 
        PULocationID, 
        'morning' AS day_time,
        ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
    FROM '2021_Yellow_Taxi_Trip_Data.csv' 
    WHERE CAST(tpep_pickup_datetime AS time) >= '08:00:00' 
          AND CAST(tpep_pickup_datetime AS time) <= '10:00:00' 
          AND passenger_count > 0 
    GROUP BY PULocationID 
),
evening_ranks AS (
    SELECT 
        COUNT(*) AS pickups, 
        PULocationID, 
        'evening' AS day_time,
        ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank
    FROM '2021_Yellow_Taxi_Trip_Data.csv' 
    WHERE CAST(tpep_pickup_datetime AS time) >= '17:00:00' 
          AND CAST(tpep_pickup_datetime AS time) <= '19:00:00' 
          AND passenger_count > 0 
    GROUP BY PULocationID 
)

SELECT pickups, PULocationID, day_time
FROM morning_ranks
WHERE rank <= 10

UNION ALL

SELECT pickups, PULocationID, day_time
FROM evening_ranks
WHERE rank <= 10

ORDER BY day_time, pickups DESC;


,pickups,PULocationID,day_time
0,18317,161,evening
1,17943,237,evening
2,16124,132,evening
3,13980,162,evening
4,13971,236,evening
5,13821,170,evening
6,12999,186,evening
7,11992,163,evening
8,11610,234,evening
9,11571,142,evening


In [ ]:
# Cell is used for grading. Do not remove.

In [8]:

%%sql 
SELECT
    tpep_pickup_datetime AS to_work_time,
    tpep_dropoff_datetime,
    PULocationID AS home_location,
    DOLocationID AS work_location,
    passenger_count
FROM
    '2021_Yellow_Taxi_Trip_Data.csv'
WHERE
    DOLocationID IN (SELECT DOLocationID FROM '2021_Yellow_Taxi_Trip_Data.csv' ORDER BY DOLocationID DESC LIMIT 1)
    AND (tpep_dropoff_datetime - tpep_pickup_datetime) < INTERVAL '60 minutes';


,to_work_time,tpep_dropoff_datetime,home_location,work_location,passenger_count
0,2021-08-11 19:32:41,2021-08-11 20:19:02,229,265,1.0
1,2021-08-11 19:34:37,2021-08-11 20:07:47,132,265,1.0
2,2021-08-11 19:09:00,2021-08-11 19:43:45,138,265,2.0
3,2021-08-11 19:05:56,2021-08-11 19:26:50,132,265,5.0
4,2021-08-11 19:58:38,2021-08-11 20:17:27,203,265,1.0
...,...,...,...,...,...
9497,2021-08-25 10:37:50,2021-08-25 11:25:16,75,265,NaN
9498,2021-08-25 11:20:00,2021-08-25 12:00:00,32,265,NaN
9499,2021-08-25 11:49:55,2021-08-25 12:12:38,48,265,NaN
9500,2021-08-25 12:40:42,2021-08-25 13:17:57,38,265,NaN


#### Q5.3: Which trips might be trips to and from work? (challenging)
Figure out which pair of trips might be people taking a taxi to and from work. The output should include all pairs of potentially matching trips. Specifically, this might mean that one "trip to work" might be matched with multiple "trip from work".

_Hint: This requires a huge join. Make reasonable assumptions, e.g. trips should be shorter than 60 mins, to reduce the amount of data that will be joined and potentially introduce some preparational steps to speed up processing. For the latter, have a look at `with` expressions in SQL!_

In [19]:
%%sql
WITH to_work AS (
    SELECT
    tpep_pickup_datetime AS to_work_time,
    tpep_dropoff_datetime,
    PULocationID AS home_location,
    DOLocationID AS work_location,
    passenger_count
FROM
    '2021_Yellow_Taxi_Trip_Data.csv'
WHERE
    HOUR(tpep_pickup_datetime) BETWEEN 6 AND 10
    AND (tpep_dropoff_datetime - tpep_pickup_datetime) < INTERVAL '60 minutes'
),
from_work AS (
    SELECT
        tpep_pickup_datetime AS from_work_time,
        tpep_dropoff_datetime,
        PULocationID AS work_location,
        DOLocationID AS home_location,
        passenger_count
    FROM
        '2021_Yellow_Taxi_Trip_Data.csv'
    WHERE
        
        HOUR(tpep_pickup_datetime) BETWEEN 16 AND 19
        and 
        AND (tpep_dropoff_datetime - tpep_pickup_datetime) < INTERVAL '60 minutes'
)

SELECT
    to_work.to_work_time,
    from_work.from_work_time,
    to_work.home_location,
    to_work.work_location,
    to_work.passenger_count
FROM
    to_work
JOIN
    from_work
ON
    to_work.work_location = from_work.work_location
    AND to_work.home_location = from_work.home_location
    AND to_work.passenger_count = from_work.passenger_count
    AND CAST(to_work.to_work_time AS DATE) = CAST(from_work.from_work_time AS DATE) 
    AND from_work.from_work_time > to_work.tpep_dropoff_datetime 
ORDER BY
    to_work.to_work_time;


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,to_work_time,from_work_time,home_location,work_location,passenger_count
0,2021-07-24 10:23:25,2021-07-24 18:19:30,186,232,1.0
1,2021-07-27 09:45:55,2021-07-27 16:07:59,237,161,1.0
2,2021-08-01 06:00:00,2021-08-01 19:43:03,186,229,1.0
3,2021-08-01 06:00:00,2021-08-01 19:23:20,186,229,1.0
4,2021-08-01 06:00:00,2021-08-01 19:24:49,186,229,1.0
...,...,...,...,...,...
5257685,2021-08-31 10:59:59,2021-08-31 17:23:24,263,262,1.0
5257686,2021-08-31 10:59:59,2021-08-31 16:14:01,263,262,1.0
5257687,2021-08-31 10:59:59,2021-08-31 16:39:48,263,262,1.0
5257688,2021-08-31 10:59:59,2021-08-31 16:33:14,263,262,1.0


## Feedback (voluntary)

How did you like this exercise? What could be improved?

Answer:

...

Further, I feel like:
 - [ ] the exercise was too easy
 - [x] the exercise was too hard
 - [ ] the exercise was just right
 - [ ] no answer
